In [ ]:
import pickle
import os
import sys
from tqdm.auto import tqdm

sys.path.insert(0, '../olympus')
sys.path.insert(0, '../graphnet/src')
sys.path.insert(0, '../gnn_testbed')
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="0.1"

import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#from tqdm.auto import tqdm

import numpy as np
import seaborn as sns
import random
from jax import random
from jax import numpy as jnp

import torch
import torch_cluster
import torch_geometric
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from nemesis.event_generation.detector import make_line, generate_noise, Detector, make_triang
from nemesis.plotting import plot_event, plot_events, plot_confusion
from nemesis.data_handling.utils import event_labelling
from nemesis.node_features.feature_generation import generate_features
from nemesis.evaluation.evaluation import model_evaluation
from nemesis.evaluation.utils import count_parameters
from nemesis.models.train import train_model
from nemesis.models.gnns import Dynamic_class, DynEdge_modified

from torch.nn import Linear, Identity, ReLU, Softmax, Dropout, LeakyReLU
import torch.nn.functional as F
from torch_geometric.nn import knn_graph, TAGConv, global_mean_pool, global_max_pool, BatchNorm,  global_add_pool, EdgeConv, DynamicEdgeConv

from graphnet.models.gnn.dynedge import DynEdge
from graphnet.components.layers import DynEdgeConv
from graphnet.models.gnn.gnn import GNN
from graphnet.models.utils import calculate_xyzt_homophily, calculate_xyz_homophily_POne

%matplotlib inline


: 

In [4]:
outpath = "."
plt.rcParams['figure.dpi'] = 100
if torch.cuda.is_available():
    print('CUDA')
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print('CPU')
    

CUDA


In [5]:
rng = np.random.RandomState(31338)
oms_per_line = 20
dist_z = 50 # m
dark_noise_rate = 16 * 1e-5  # 1/ns
side_len = 100 # m
pmts_per_module = 16
pmt_cath_area_r = 75E-3 / 2 # m
module_radius = 0.21 # m
v_x = 0
efficiency = pmts_per_module * (pmt_cath_area_r)**2 * np.pi / (4*np.pi*module_radius**2)
det = make_triang(side_len, oms_per_line, dist_z, dark_noise_rate, rng, efficiency=efficiency, v_x=v_x, buoy_weight=30)
module_positions = jnp.asarray(det.module_coords)

In [4]:
#from multiprocessing import Pool
#import pickle
#import numpy as np
#import os
#file_idx = range(12)

def load_all_events(it):
    path_to_events = "/dss/pone/pone_events/all_events"
    load_array = ['cascades_15000ev_3.5-6.5_1.pickle',  'cascades_15000ev_3.5-6.5_2.pickle',
                  'cascades_15000ev_3.5-6.5_3.pickle',  'cascades_15000ev_3.5-6.5_4.pickle',
                  'stracks_15000ev_3.5-6.5_1.pickle',   'stracks_15000ev_3.5-6.5_2.pickle', 
                  'stracks_15000ev_3.5-6.5_3.pickle',   'stracks_15000ev_3.5-6.5_4.pickle', 
                  'tracks_15000ev_3.5-6.5_1.pickle',    'tracks_15000ev_3.5-6.5_2.pickle', 
                  'tracks_15000ev_3.5-6.5_3.pickle',    'tracks_15000ev_3.5-6.5_4.pickle']
            
    events, records = pickle.load(open(os.path.join(path_to_events, load_array[it]), "rb"))
    
    print(f'{it} loaded!')
    
    return events, records

In [6]:
#pickle.dump(data_array, open("/dss/pone/pone_events/features_arrays/data_array_180k_k8_Rout_right.pickle", "wb"))
data_array = pickle.load(open("/dss/pone/pone_events/features_arrays/data_array_180k_k8_Rout.pickle", "rb"))

KeyboardInterrupt: 

In [12]:
import random
random.seed(81722)

indices = np.arange(len(data_array))
random.shuffle(indices)


shuffled_data = [data_array[i] for i in indices]
split_test = int(len(shuffled_data)*0.9)
training_data = shuffled_data[:split_test]
split_val = int(len(training_data)*0.9)
train_dataset = training_data[:split_val]
val_dataset = training_data[split_val:]

test_dataset = shuffled_data[split_test:]
test_indices = indices[split_test:]

In [24]:
import torch
from torch import Tensor
from torch.nn import Linear, Identity, ReLU, BatchNorm1d, LeakyReLU
import torch.nn.functional as F
from torch_geometric.nn import TAGConv, global_max_pool, BatchNorm, DynamicEdgeConv
from typing import Any, Dict, List, Optional, Union
from torch_geometric.typing import NoneType
from torch_scatter import scatter_max, scatter_mean, scatter_min, scatter_sum

from graphnet.models.gnn.gnn import GNN
from graphnet.components.layers import DynEdgeConv
from nemesis.models.utils import calculate_xyz_homophily_POne

In [25]:
class DynEdge_modified(GNN):
    def __init__(self, input_features, output_features, k = 15, features_subset = slice(12, 15), layer_size_scale=4):
        
        #Architecture configuration
        c = layer_size_scale
        l1, l2, l3, l4, l5, l6, l7 = (
            input_features, c * 16 * 2, c * 32 * 2, c * 64 * 2, c * 32 * 2, c * 512 * 2, output_features)
        
        #Base class constructor
        super().__init__(l1, l6)
        
        #First Layer
        self.conv_add1 = DynEdgeConv(
            torch.nn.Sequential(Linear(l1*2, l2),LeakyReLU(),Linear(l2, l3),LeakyReLU()),
            aggr="max",
            nb_neighbors=k,
            features_subset=features_subset)              
          
        #Second Layer  
        self.conv_add2 = DynEdgeConv(
            torch.nn.Sequential(Linear(l3*2, l4), LeakyReLU(), Linear(l4, l3), LeakyReLU()),
            aggr="add",
            nb_neighbors=k,
            features_subset=features_subset)
        
        #Third Layer
        self.conv_add3 = DynEdgeConv(
            torch.nn.Sequential(Linear(l3*2, l4),LeakyReLU(),Linear(l4, l3),LeakyReLU()),
            aggr="max",
            nb_neighbors=k,
            features_subset=features_subset)
        
        #Fourth Layer
        self.conv_cat1 = DynEdgeConv(
            torch.nn.Sequential(Linear(l3*2, l4),LeakyReLU(),Linear(l4, l3),LeakyReLU()),
            aggr="add",
            nb_neighbors=k,
            features_subset=features_subset)
        
        #Linear layers
        self.nn1 = Linear(l3*4 + l1, l4)
        self.nn2 = Linear(l4, l5)
        self.nn3 = Linear(4*l5 + 3, l6)
        self.lrelu = LeakyReLU()
        
        self.mlp = MLP([l6, l6/2, l6/8, l6/32, l7], dropout=0.5)
        
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        h_x, h_y, h_z = calculate_xyz_homophily_POne(x, edge_index, batch)
        
        global_means = scatter_mean(x, batch, dim=0)
        
        a, edge_index = self.conv_add1(x, edge_index, batch)
        b, edge_index = self.conv_add2(a, edge_index, batch)
        c, edge_index = self.conv_add3(b, edge_index, batch)
        d, edge_index = self.conv_cat1(c, edge_index, batch)
        
        #Skip cat
        x = torch.cat((x, a, b, c, d), dim=1)
        
        #Post-processing
        x = self.nn1(x)
        x = self.lrelu(x)
        x = self.nn2(x)
        
        #Aggregation across nodes
        a, _ = scatter_max(x, batch, dim=0)
        b, _ = scatter_min(x, batch, dim=0)
        c = scatter_sum(x, batch, dim=0)
        d = scatter_mean(x, batch, dim=0)
        
        #Cat aggr and scalar feats
        
        x = torch.cat((a, b, c, d, h_x, h_y, h_z), dim=1)
        
        #Readout
        
        x = self.lrelu(x)
        x = self.nn3(x)
        
        x = self.lrelu(x)
        
        return x     

In [23]:
model = DynEdge_modified(15, 4, k = 8, features_subset = slice(12, 15), layer_size_scale=4)
model.to(device)
count_parameters(model)

label_map = {0:"Contained cascade", 1:'Throughgoing Track', 2:"Starts in detector", 3:"Rest of events"}
model, all_trains_acc, all_vals_acc = train_model(model, train_dataset, val_dataset, label_map, k=15, epochs=250, patience=20, print_step=1, batch_size=200, lr=0.001)
plt.plot(np.linspace(0, len(all_trains_acc), len(all_trains_acc)), all_trains_acc, label="Training Accuracy")
plt.plot(np.linspace(0, len(all_vals_acc), len(all_vals_acc)), all_vals_acc, label="Validation Accuracy")
plt.legend()
plt.show()


NameError: name 'MLP' is not defined

In [7]:
test_loader = DataLoader(test_dataset, batch_size=250, shuffle=False)
fig1, miss_idx1, test_acc=model_evaluation(model, test_loader, all_events, all_records, all_labels, label_map, test_indices)
print('The test accuracy achieved is: ', test_acc)

NameError: name 'test_dataset' is not defined

In [18]:
path_to_models = "/dss/pone/nemesis_trained_models/"
#model = torch.load(os.path.join(outpath, "Dynmod_model_180k_R_75.pt"))
torch.save(model, os.path.join(outpath, "DynMod_model_180k_R_out_B250_107E_L8.pt"))

In [ ]:
for i,ii in enumerate(data_array):
    if i%1000 == 0:
        print(ii.y)

In [ ]:
import pickle
import os
import functools
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="0.8"
import sys
sys.path.insert(0, '../olympus')
sys.path.insert(0, '../hyperion')
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from itertools import product
import awkward as ak
import pandas as pd
from olympus.event_generation.photon_propagation.norm_flow_photons import make_generate_norm_flow_photons, make_nflow_photon_likelihood
from olympus.event_generation.photon_propagation.utils import sources_to_model_input
from nemesis.event_generation.detector import Detector, make_line, make_triang
from olympus.event_generation.event_generation import (
    generate_cascade,
    generate_cascades,
    simulate_noise,
    generate_realistic_track,
    generate_realistic_tracks,
    generate_realistic_tracks_test,
    generate_realistic_starting_tracks,)
from olympus.event_generation.lightyield import make_pointlike_cascade_source, make_realistic_cascade_source
from olympus.event_generation.utils import sph_to_cart_jnp, proposal_setup

#from olympus.plotting import plot_event
from hyperion.medium import medium_collections
from hyperion.constants import Constants
import jax
jax.config.update('jax_platform_name', 'cpu')
from jax import random
from jax import numpy as jnp
import json

path_to_config = "../hyperion/data/pone_config_optimistic.json"
config = json.load(open(path_to_config))["photon_propagation"]
ref_ix_f, sca_a_f, sca_l_f, _ = medium_collections[config["medium"]]

def c_medium_f(wl):
    """Speed of light in medium for wl (nm)."""
    return Constants.BaseConstants.c_vac / ref_ix_f(wl)

rng = np.random.RandomState(31338)
oms_per_line = 20
dist_z = 50 # m
dark_noise_rate = 16 * 1e-5  # 1/ns
side_len = 100 # m
pmts_per_module = 16
pmt_cath_area_r = 75E-3 / 2 # m
module_radius = 0.21 # m



In [5]:
def load_test_events(v_x):
    path_to_events = "/dss/pone/pone_events"
    det, cascades, cascade_records = pickle.load(open(os.path.join(path_to_events, f"cascades1500_vx{v_x}.pickle"), "rb"))
    det, tracks, track_records = pickle.load(open(os.path.join(path_to_events, f"tracks1500_vx{v_x}.pickle"), "rb"))
    det, stracks, strack_records = pickle.load(open(os.path.join(path_to_events, f"starting_tracks1500_vx{v_x}.pickle"), "rb"))    
    return det, cascades, cascade_records, tracks, track_records, stracks, strack_records
    
cascades_test, cascade_records_test = [], []
tracks_test, track_records_test = [], []
stracks_test, strack_records_test = [], []
det_test = []
v_x_all = []
for it in range(1,21):
    v_x_all.append(np.round(it*0.005, 3))

In [ ]:
v_x_all = []
for it in range(1,21):
    v_x_all.append(np.round(it*0.005, 3))
pbar = tqdm(total=len(v_x_all))

for v_x in v_x_all:
    det, cascades, cascade_records, tracks, track_records, stracks, strack_records = load_test()
    cascades_test += cascades
    cascade_records_test += cascade_records
    tracks_test += tracks
    track_records_test += track_records
    stracks_test += stracks
    strack_records_test += strack_records
    pbar.update()

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [5]:
from nemesis.data_handling.event_loading import load_test

det_test, cascades_test, cascade_records_test, tracks_test, track_records_test, stracks_test, strack_records_test = load_test()

  0%|          | 0/20 [00:00<?, ?it/s]

0.005 speed loaded!
0.01 speed loaded!
0.015 speed loaded!
0.02 speed loaded!
0.025 speed loaded!
0.03 speed loaded!
0.035 speed loaded!
0.04 speed loaded!
0.045 speed loaded!
0.05 speed loaded!
0.055 speed loaded!
0.06 speed loaded!
0.065 speed loaded!
0.07 speed loaded!
0.075 speed loaded!
0.08 speed loaded!
0.085 speed loaded!
0.09 speed loaded!
0.095 speed loaded!
0.1 speed loaded!


In [10]:
cascade_labels_test, track_labels_test, strack_labels_test = event_labelling(track_records_test, strack_records_test, cascade_records_test, det_hull=det.outer_cylinder)
all_events_test = cascades_test + tracks_test + stracks_test
all_records_test = cascade_records_test + track_records_test + strack_records_test
all_labels_test = cascade_labels_test + track_labels_test + strack_labels_test
all_records_test = pickle.load(open("/dss/pone/pone_events/features_arrays/test_data_array/all_records_test_90k_k8_R75_v_x.pickle", "rb"))


In [11]:
from nemesis.evaluation.evaluation import model_evaluation, energy_division_loaders, energy_evaluation

test_energy_loaders = energy_division_loaders(det, all_events_test, all_labels_test, all_records_test, num_divisions=11, k=15, batch_size=250)

  0%|          | 0/90000 [00:00<?, ?it/s]

In [12]:
pickle.dump(test_energy_loaders, open("/dss/pone/pone_events/features_arrays/test_data_array/test_energy_loaders_90k_k15_Rout_B250_11divisions.pickle", "wb"))

In [13]:
cascade_labels_test, track_labels_test, strack_labels_test = event_labelling(track_records_test, strack_records_test, cascade_records_test, det_hull=(100,1000))
all_events_test = cascades_test + tracks_test + stracks_test
all_records_test = cascade_records_test + track_records_test + strack_records_test
all_labels_test = cascade_labels_test + track_labels_test + strack_labels_test
all_records_test = pickle.load(open("/dss/pone/pone_events/features_arrays/test_data_array/all_records_test_90k_k8_R75_v_x.pickle", "rb"))


In [ ]:
test_energy_loaders = energy_division_loaders(det, all_events_test, all_labels_test, all_records_test, num_divisions=11, k=15, batch_size=250)

In [ ]:
pickle.dump(test_energy_loaders, open("/dss/pone/pone_events/features_arrays/test_data_array/test_energy_loaders_90k_k15_R100_B250_11divisions.pickle", "wb"))

In [23]:
cascade_labels_test, track_labels_test, strack_labels_test = event_labelling(track_records_test, strack_records_test, cascade_records_test, det_hull=(100,1000))
all_events_test = cascades_test + tracks_test + stracks_test
all_records_test = cascade_records_test + track_records_test + strack_records_test
all_labels_test = cascade_labels_test + track_labels_test + strack_labels_test
all_records_test = pickle.load(open("/dss/pone/pone_events/features_arrays/test_data_array/all_records_test_90k_k8_R75_v_x.pickle", "rb"))


In [24]:
pickle.dump(test_energy_loaders, open("/dss/pone/pone_events/features_arrays/test_data_array/test_energy_loaders_90k_k15_R100_B250_11divisions.pickle", "wb"))

In [25]:
test_energy_loaders

In [85]:
#pickle.dump(data_array, open("/dss/pone/pone_events/features_arrays/data_array_test_90k_k8_R75_v_x.pickle", "wb"))
#pickle.dump(all_records_test_info, open("/dss/pone/pone_events/features_arrays/all_records_test_90k_k8_R75_v_x.pickle", "wb"))

In [ ]:
all_records_test_info = pickle.load(open("/dss/pone/pone_events/features_arrays/all_records_test_90k_k8_R75_v_x.pickle", "rb"))
test_energy_loaders = energy_division_loaders(det, all_events_test, all_labels_test, all_records_test, num_divisions=11, k=15, batch_size=250)

In [7]:
model = torch.load("/dss/pone/nemesis_trained_models/Mar12/DynEdgeMod_180.0k_R100_k11_B200_L1_lr0.001.pt")
test_energy_loaders = pickle.load(open("/dss/pone/pone_events/features_arrays/test_data_array/test_energy_loaders_90k_k15_R75.pickle", "rb"))

In [15]:
fig, test_accuracies, misscls_idx = energy_evaluation(model, test_energy_loaders)

  0%|          | 0/90000 [00:00<?, ?it/s]

first half done
second half done
first half done
second half done
first half done
second half done
first half done
second half done
first half done
second half done
first half done
second half done
first half done
second half done
first half done
second half done
first half done
second half done
The accuracies of the different energy divisions are [33.39610501702682, 36.014179683172706, 36.80731090803033, 37.82696177062374, 40.0, 42.359382777501246, 45.25139664804469, 41.37670196671709, 47.96680497925311]


In [15]:
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns
from nemesis.node_features.feature_generation import get_features
from torch_geometric.loader import DataLoader
from torch_geometric.nn import knn_graph

def energy_division_loaders(det, all_events_test, all_labels_test:list, all_records_test:dict, num_divisions:int, k=8, batch_size=200,log_emin=None, log_emax=None):
    #Create the energy bins
    if log_emin and log_emax:
        #Emin = np.power(10, log_emin)
        #Emax = np.power(10, log_emax)
        energy_divisions = np.logspace(log_emin, log_emax, num_divisions)
        #energy_divisions = np.linspace(Emin, Emax, num_divisions)
    else:
        all_energies = [sub['energy'] for sub in all_records_test]
        energy_divisions = np.linspace(np.min(all_energies), np.max(all_energies), num_divisions)

    len_divisions = 0
    #Create dict with the index of the events for each energy division
    interval_dict = {}

    for interval_nr in range(len(energy_divisions)-1):
        interval_dict[f"interval{interval_nr}"] = []
        count = 0
        for nr,rc in enumerate(all_records_test):
            if rc['energy'] >= energy_divisions[interval_nr] and rc['energy'] < energy_divisions[interval_nr+1]:
                count+=1
                #print(energy_divisions[interval_nr+1])
                interval_dict[f"interval{interval_nr}"].append(nr)
        len_divisions+=count

    pbar = tqdm(total=len(all_labels_test))
    test_energy_loaders = []

    #Generate the test energy loaders for each energy division
    for int_nr, interval in enumerate(interval_dict):
        data_array = []
        for ev in interval_dict[interval]:
            features = get_features(det, all_events_test[ev])
            valid = np.all(np.isfinite(features), axis=1)
            features = features[valid]
            x = torch.Tensor(features)

            edge_index = knn_graph(x[:, [-1, -2, -3]], k=k, loop=False)
            #data = transf(torch_geometric.data.Data(x, edge_index, y=torch.tensor([label], dtype=torch.int64)).to(device))
            data = torch_geometric.data.Data(x, edge_index, y=torch.tensor(all_labels_test[ev], dtype=torch.int64)).to(device)
            data_array.append(data)
            pbar.update()
        loader = DataLoader(data_array, batch_size, shuffle=False)
        test_energy_loaders.append(loader)
        
    return test_energy_loaders

def energy_evaluation(model, test_energy_loaders, plot_conf=True, label_map = {0:"Contained cascade", 1:'Throughgoing Track', 2:"Starts in detector", 3:"Rest of events"}):    

    nplt = int(np.ceil(np.sqrt(len(test_energy_loaders))))
    fig = plt.figure(figsize=(nplt * 4, nplt * 4))
    pbar = tqdm(total=90000)
    test_accuracies = []

    ## Checking for predictions
    with torch.no_grad():
        for i, loader in enumerate(test_energy_loaders):
            test_preds = []
            test_truths = []
            test_scores = []
            all_preds_test_len = 0

            for data in loader:  # Iterate in batches over the training/test dataset.
                out = model(data)
                #print(out.shape)
                test_pred = out.argmax(dim=1)
                test_preds.append(test_pred)
                test_truths.append(data.y)
                test_scores.append(out)
                all_preds_test_len+=len(data.y)
                pbar.update()

            test_preds = torch.cat(test_preds).cpu()
            test_truths = torch.cat(test_truths).cpu()
            test_scores = torch.cat(test_scores).cpu()
            misscls_idx = np.atleast_1d(np.argwhere(test_preds != test_truths).ravel())
            test_accuracy = 100 * (all_preds_test_len - len(misscls_idx)) / all_preds_test_len
            all_preds = []
            
            for j in range(len(label_map)):
                true_sel = test_truths == j
                predictions = np.histogram(test_preds[true_sel], bins=np.arange(0, len(label_map)+1, 1))[0]
                if predictions.sum() == 0:
                    predictions = predictions
                else:
                    predictions = predictions / predictions.sum()
                all_preds.append(predictions)

            all_preds = np.vstack(all_preds)
            test_accuracies.append(test_accuracy)
            
            if plot_conf:
                ax = fig.add_subplot(nplt, nplt, i + 1)
                ax1 = sns.heatmap(all_preds, cmap=plt.cm.Blues, annot=True, ax=ax)# xticklabels=list(label_map.values()), yticklabels=list(label_map.values()), ax=ax)
                ax1.set_xlabel("Predicted Label")
                ax1.set_ylabel("True Label")
            #ax1.set_title()
            #fig1 = ax.get_figure()
            
            ax, ax1 = 0,0


    plt.show()
    print('The accuracies of the different energy divisions are', test_accuracies)

    return fig, test_accuracies, misscls_idx